In [1]:
cd ..

/Users/vijay/vijaygkd/char-rnn


In [2]:
%load_ext autoreload
%autoreload 2

# Imports

In [101]:
import numpy as np

import torch
import torch.nn as nn
from torchinfo import summary

In [102]:
from data.data_utils import process_corpus, str_to_ids, char_idx_to_str
from data.dataloader import process_input_file

from train.train import train, test_batch_gen_text

from model.lstm import LSTM_CharLM
from model.model_utils import sample_token

# Model Exp

## 1. LSTM

In [13]:
m = LSTM_CharLM(128)

summary(m, input_size=(1,50), dtypes=[torch.long])

Layer (type:depth-idx)                   Output Shape              Param #
LSTM_CharLM                              [1, 50, 128]              --
├─Embedding: 1-1                         [1, 50, 12]               1,536
├─LSTM: 1-2                              [1, 50, 128]              72,704
├─Linear: 1-3                            [1, 50, 128]              16,512
Total params: 90,752
Trainable params: 90,752
Non-trainable params: 0
Total mult-adds (M): 3.65
Input size (MB): 0.00
Forward/backward pass size (MB): 0.11
Params size (MB): 0.36
Estimated Total Size (MB): 0.47

In [100]:
# initial value of loss function for untrained model.
# loss = cross entropy for randomly selecting a token from char set of size 128
-np.log(1/128)

4.852030263919617

### Toy sample
Overfit 1 data point

In [113]:
roger = """Roger Federer is widely considered to be one of the greatest tennis players of all time. He has won a record 20 Grand Slam singles titles, including 8 Wimbledon, 6 Australian Open, 5 US Open, and 1 French Open titles. He has also won the ATP Tour Finals a record 6 times, and has been ranked world No. 1 for a record 310 weeks, including 237 consecutive weeks from 2004 to 2008. Federer's all-around game, characterized by his exceptional shot-making, movement, and court coverage, has been called one of the greatest examples of "complete" tennis. He is also known for his longevity, with many of his records coming in his 30s and after. In addition to his success on the court, Federer is also known for his sportsmanship, and is widely respected by both fans and fellow players. Off the court, he is a philanthropist and has been involved in various charitable organizations throughout his career."""

In [123]:
#data
# corpus = [
#     "This text is written by Vijay Gaikwad :)",
#     "Ruchira is my beautiful wife.",
#     "We both happily live in Seattle."
# ]

corpus = [roger[:200]]


X, Y = process_corpus(corpus, seq_len=len(corpus[0])+1)

print(X.shape)

torch.Size([1, 201])


In [124]:
m = LSTM_CharLM(8)

In [125]:
train(m, X, Y, X_test=X, epochs=3000, lr=0.01)

  0%|▎                                                                                              | 9/3001 [00:00<01:12, 41.49it/s]

-----------------
Epoch: 0 | Loss: 4.967576503753662
} WxWWWxxWxW WW.xWWWWWWWWWWWWWWWWWW}xWWW}WW}}xWWWWWWWWWW.WWWWWWWWWW W}WW}WWWWxWWW}}}WWWWW WW .WWWWWx}Wx}WWWWWW}WWWWWWWWW}WWWWWW


 17%|███████████████▊                                                                             | 509/3001 [00:10<00:53, 46.57it/s]

-----------------
Epoch: 500 | Loss: 1.2563029527664185
Roger tldere  ts pinere ton  sele  tinoe oferon tie ore nld loid in tlayers tf tle tine. te oam ton tncedone t0 crand O ameiisg ed tinled  tscous ng o tisbleden  t tusilelenn Open, 5 5S tpen, tnd O tl$


 34%|██████████████████████████████▉                                                             | 1009/3001 [00:22<00:46, 43.06it/s]

...

 67%|█████████████████████████████████████████████████████████████▍                              | 2006/3001 [00:46<00:26, 37.14it/s]

-----------------
Epoch: 2000 | Loss: 0.20960144698619843
Roger Federer is widely considere  ti be one of the greatest tennis players of all time. He has won a record 20 Grand Slam singles titles, including 8 Wimbledon, 6 Austrelean Open, 5 US Open, and 1 Fr$


 83%|████████████████████████████████████████████████████████████████████████████▊               | 2504/3001 [00:59<00:15, 32.45it/s]

-----------------
Epoch: 2500 | Loss: 0.3797396421432495
Roger Federer is pidely cons dere  ti be ofe of tre oreatls eeennis wlayers if all time. He han win a iecors 20 Grand Slam tingles titles, including 8 Wimbledon, 6 tustreledn Open, 5 US Open, and S Ar$


-----------------
Epoch: 3000 | Loss: 0.10870976001024246
Roger Federer is widely considered to be one of the greatest tennis players of all time. He has won a record 20 Grand Slam singles titles, including 8 Wimbledon, 6 Australean Open, 5 US Open, and 1 Fr$


In [126]:
sample_token(m)

'$Roger Federer is widely considere gles 20 Grand Slam singles titles, including 8 Wimbledo 8 Wimbledo'

### Toy Coffee Dataset

Toy coffee corpus containing 64+8 englist sentences about coffee generated by godfather ChatGPT. 
Model is trained to 64 sentences, remaining 8 are used to test the model's language ability.

In [109]:
f = 'data_files/coffee.txt'
train_dl, val_dl = process_input_file(f, seq_len=50, batch_size=16)

Train: samples=107 batches=7 | Val: samples=5 batches=1


In [111]:
# Train model.
m = LSTM_CharLM(64, hidden_layers=2, dropout=0.5)

train(m, train_dl, val_dl, epochs=1000, lr=0.01)

  0%|▏                                                                                                | 2/1001 [00:00<02:48,  5.94it/s]

-----------------
Epoch: 0 | Loss: 3.978226287024362 | Perplexity: 53.42219450355294
Epoch: 0 | Val Loss: 3.204084873199463 | Perplexity: 24.63294742842331
<Target>: e aroma of freshly brewed coffee was a perfect way
<Prediction>:                                                   


 10%|█████████▋                                                                                     | 102/1001 [00:15<02:34,  5.83it/s]

-----------------
Epoch: 100 | Loss: 0.8907552531787327 | Perplexity: 2.4369694853824564
Epoch: 100 | Val Loss: 0.9740163087844849 | Perplexity: 2.6485605633646765
<Target>: e aroma of freshly brewed coffee was a perfect way
<Prediction>:  anoma of freshly brewed coffee aas a serfect pay 


 20%|███████████████████▏                                                                           | 202/1001 [00:32<02:24,  5.53it/s]

...

 80%|████████████████████████████████████████████████████████████████████████████                   | 801/1001 [02:37<00:43,  4.64it/s]

-----------------
Epoch: 800 | Loss: 0.2484269972358431 | Perplexity: 1.2820072288816826
Epoch: 800 | Val Loss: 1.3739206790924072 | Perplexity: 3.9508102287947335
<Target>: e aroma of freshly brewed coffee was a perfect way
<Prediction>:  wnoma of freshly brewed coffee aas a perfect pay 


 90%|█████████████████████████████████████████████████████████████████████████████████████▌         | 901/1001 [02:58<00:22,  4.54it/s]

-----------------
Epoch: 900 | Loss: 0.20885350448744638 | Perplexity: 1.2322644640925664
Epoch: 900 | Val Loss: 1.2980910539627075 | Perplexity: 3.662298859629902
<Target>: o have a cup.
The barista skillfully poured steame
<Prediction>: faameeancup.
The barista ckillfully poured steamed


-----------------
Epoch: 1000 | Loss: 0.23026093201977865 | Perplexity: 1.2589284618221066
Epoch: 1000 | Val Loss: 1.3836787939071655 | Perplexity: 3.9895514016115197
<Target>: a morning pick-me-up.
They sat outside, sipping th
<Prediction>: spyrning pick-me-up 
The  sat$outside. sipping the


In [112]:
context = "The coffee"
sample_token(coffee_model, context, max_len=200)

"$The coffeee woy to start the dar.\nHe couldn't resicut fuld the tarerr and culan on and ensle bup of coffee while resperfect way to start the day.\nHe couldn't help the roasted and had and treat fromh nor the pes"